### 神奈川中央交通本厚木駅と小田急本厚木駅のように近いが物理的に接続されているわけではない駅同士を、徒歩数分圏内という区切りで接続するための実験  
bus_and_train.ipynbが一番似てる

In [3]:
from engine import Station


# Stationの座標を用いて近くの交通機関を取得する
def near_transport_station(reference: Station, candidates: list[Station]) -> list[Station]:
    near_distance_threshold = 800  # 近所とみなす距離
    nears: list[Station] = []

    for candidate in candidates:
        distance = calc_distance_m(
            reference.geometry.calc_mean(),  # 平均値で計算する
            candidate.geometry.calc_mean(),
        )
        if distance <= near_distance_threshold:
            nears.append(candidate)

    return nears

In [4]:
from typing import Optional
from engine import TransitType
from engine.bus import *
from engine.train import *

# データの読み込み
dataset: dict[TransitType, list[Station]] = {
    TransitType.BUS: load_stop_data("../dataset/busstops/kanagawa/P11-22_14.geojson"),
    TransitType.TRAIN: load_station_data("../dataset/stations/N02-20_Station.geojson"),
}

# バスの本厚木駅を取り出す
honatugi_bus_stop: Optional[Station] = None
for stop in dataset[TransitType.BUS]:
    if stop.name == "本厚木駅":
        honatugi_bus_stop = stop
if honatugi_bus_stop is None:
    print("本厚木駅をみつけられませんでした")

# バスの本厚木駅から近い電車の駅を探す
near_stations = near_transport_station(honatugi_bus_stop, dataset[TransitType.TRAIN])
for near_station in near_stations:
    print(", ".join(near_station.management_groups), near_station.name)


小田急電鉄 本厚木
